In [3]:
import os
import openai

from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.agents import tool

from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate


In [4]:
from langchain.schema import BaseRetriever, Document


In [5]:
api_type = "azure"
api_base_url = "https://openaidemonubiral.openai.azure.com/"
api_version = "2023-03-15-preview"
azure_api_key = "ff5c606c134e4d1dae3426a412df834a"

openai.api_type = api_type
openai.api_base = api_base_url
openai.api_version = api_version
openai.api_key = azure_api_key

os.environ["OPENAI_API_BASE"] = api_base_url
os.environ["OPENAI_API_KEY"] = azure_api_key

In [6]:
embeddings_azure = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

llm_azure_chatgpt = AzureChatOpenAI(
    deployment_name="nubiral-lab-01", 
    temperature=0, 
    openai_api_version=api_version)

#nubiral-lab-01

In [7]:
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "cognitive-searchdemo"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "cosmosdbfinance-index"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "Ndqr05h9fitPrfEh9GxNBb2vsvelw9oUgIew1VpUvYAzSeD2n9Zi"


In [8]:
#os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "demofinance"
#os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "azureblob-index"
#os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "8kxHbImAINjTkVEW45AYc3EzlSNuFv1cqXPH6RWJ0oAzSeBdBmor"

In [9]:
pip show langchain


Name: langchain
Version: 0.0.187
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/carolina/Descargas/nu0002_qa_apps_azure-main (copia)/venv/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: llama-index
Note: you may need to restart the kernel to use updated packages.


In [10]:
retriever_cognitive_search= AzureCognitiveSearchRetriever(content_key="descripcion")

In [11]:
retriever_cognitive_search.get_relevant_documents("clientes con deudas acumuladas")

[Document(page_content='El usuario con numero de ID 00855294 tiene un saldo actual de 130000$ e ingresos mesuales de 270000$. Contra eso, se registran gastos mensuales de 220000$ lo cual lleva a un neto de 50000$, la deuda acumulada es 60000$. El usuario tiene una clasificacion de historial crediticio Malo. En terminos de actividad, se registra la ultima actualizacion el dia 2023-10-13, indica 46 dias de inactividad.', metadata={'@search.score': 0.045163043, 'usuario_id': 855294, 'saldo_actual': 130000, 'ingresos_mensuales': 270000, 'gastos_mensuales': 220000, 'deuda_total': 60000, 'historial_crediticio': 'Malo', 'dias_ultima_actualizacion': 46, 'deuda_document': 'Usuario ID: 855294\nDeuda Total: 60000\nHistorial Crediticio: Malo\nDescripción: El usuario con numero de ID 00855294 tiene un saldo actual de 130000$ e ingresos mesuales de 270000$. Contra eso, se registran gastos mensuales de 220000$ lo cual lleva a un neto de 50000$, la deuda acumulada es 60000$. El usuario tiene una clasi

In [12]:
#pip install pydantic


In [15]:
#pip install langchain==0.0.308
from langchain.schema import BaseRetriever, Document
from typing import List


class CustomRetrieverFinanceData(BaseRetriever):
    
    def __init__(self, retriever_cognitive_search, k=10):
        
        self.retriever_cognitive_search = retriever_cognitive_search
        self.k = k
        
    def get_relevant_documents(self, query: str) -> List[Document]:
        
        documents =  self.retriever_cognitive_search.get_relevant_documents(query)
        
        if len(documents) > self.k:
            documents = documents[:self.k]
        
        for document in documents:
            
            document.page_content = self.add_sources_to_document(document)
        
        return documents
    
    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError
        
    def add_sources_to_document(self, input_document):
        
        page_content_with_sources = f"Usuario_id: {input_document.metadata['usuario_id']}\nSaldo_actual: {input_document.metadata['saldo_actual']}\nIngresos_mensuales: {input_document.metadata['ingresos_mensuales']}\nGastos_mensuales: {input_document.metadata['gastos_mensuales']}\nDeuda_total: {input_document.metadata['deuda_total']}\nDias_ultima_actualizacion: {input_document.metadata['dias_ultima_actualizacion']}\nCategories: {input_document.metadata['historial_crediticio']}\nDescription: {input_document.page_content}"
        
        return page_content_with_sources

In [17]:
custom_retriever = CustomRetrieverFinanceData(retriever_cognitive_search,k=10)


In [18]:
custom_retriever.get_relevant_documents("malo")

[Document(page_content='Usuario_id: 17216\nSaldo_actual: 730000\nIngresos_mensuales: 420000\nGastos_mensuales: 360000\nDeuda_total: 0\nDias_ultima_actualizacion: 6\nCategories: Malo\nDescription: El usuario con numero de ID 00017216 tiene un saldo actual de 730000$ e ingresos mesuales de 420000$. Contra eso, se registran gastos mensuales de 360000$ lo cual lleva a un neto de 60000$, la deuda acumulada es 0$. El usuario tiene una clasificacion de historial crediticio Malo. En terminos de actividad, se registra la ultima actualizacion el dia 2023-11-22, indica 6 dias de inactividad.', metadata={'@search.score': 5.974925, 'usuario_id': 17216, 'saldo_actual': 730000, 'ingresos_mensuales': 420000, 'gastos_mensuales': 360000, 'deuda_total': 0, 'historial_crediticio': 'Malo', 'dias_ultima_actualizacion': 6, 'deuda_document': 'Usuario ID: 17216\nDeuda Total: 0\nHistorial Crediticio: Malo\nDescripción: El usuario con numero de ID 00017216 tiene un saldo actual de 730000$ e ingresos mesuales de 

In [ ]:
#pip install -U langchain

In [ ]:
#pip install langchain 

In [19]:
prompt_template = """Use the following context elements to answer the question at the end.
You are a fintech recommendation system, when asked by the user, you must recommend the clients that best meet the requirements to grant credits, in addition to that, try to provide reasons why the user should grant credits to the clients you recommend.
You can only recommend the clients provided below, if none of the clients meet the credit requirements, please indicate that you do not have something that meets the requirements. Don't try to make up an answer.


Clients:

{context}

Question: {question}
Answer in Spanish:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [20]:
chain_type = "stuff"
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llm_azure_chatgpt, chain_type=chain_type, retriever=custom_retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [21]:
query = "puedes recomendarme clientes sin deuda?"
result = qa({"query": query})

In [22]:
result['result']

'Sí, puedo recomendarte los siguientes clientes que no tienen deuda: Usuario_id 189024, Usuario_id 441701, Usuario_id 665090, Usuario_id 359464, Usuario_id 55473, Usuario_id 696286, Usuario_id 304542 y Usuario_id 212184. Todos ellos tienen un saldo actual positivo y gastos mensuales menores a sus ingresos mensuales, lo que indica una buena capacidad de pago. Además, tienen una clasificación de historial crediticio de Excelente, Bueno o Regular, lo que sugiere que han tenido un buen comportamiento en el pasado en cuanto a sus obligaciones crediticias.'

In [23]:
result['source_documents']

[Document(page_content='Usuario_id: 855294\nSaldo_actual: 130000\nIngresos_mensuales: 270000\nGastos_mensuales: 220000\nDeuda_total: 60000\nDias_ultima_actualizacion: 46\nCategories: Malo\nDescription: El usuario con numero de ID 00855294 tiene un saldo actual de 130000$ e ingresos mesuales de 270000$. Contra eso, se registran gastos mensuales de 220000$ lo cual lleva a un neto de 50000$, la deuda acumulada es 60000$. El usuario tiene una clasificacion de historial crediticio Malo. En terminos de actividad, se registra la ultima actualizacion el dia 2023-10-13, indica 46 dias de inactividad.', metadata={'@search.score': 0.05423598, 'usuario_id': 855294, 'saldo_actual': 130000, 'ingresos_mensuales': 270000, 'gastos_mensuales': 220000, 'deuda_total': 60000, 'historial_crediticio': 'Malo', 'dias_ultima_actualizacion': 46, 'deuda_document': 'Usuario ID: 855294\nDeuda Total: 60000\nHistorial Crediticio: Malo\nDescripción: El usuario con numero de ID 00855294 tiene un saldo actual de 130000$

In [ ]:
query = "necesito encontrar clientes con deuda e historial creditico excelente"
result = qa({"query": query})
result

{'query': 'necesito encontrar clientes con deuda e historial creditico excelente',
 'result': 'Lo siento, no tengo algo que cumpla con los requisitos de tener deuda e historial crediticio excelente.',
 'source_documents': [Document(page_content='El usuario con numero de ID 00793475 tiene un saldo actual de 900000$ e ingresos mesuales de 450000$. Contra eso, se registran gastos mensuales de 400000$ lo cual lleva a un neto de 50000$, la deuda acumulada es 0$. El usuario tiene una clasificacion de historial crediticio Excelente. En terminos de actividad, se registra la ultima actualizacion el dia 2023-11-23, indica 5 dias de inactividad.', metadata={'@search.score': 7.001548, 'usuario_id': 793475, 'saldo_actual': 900000, 'ingresos_mensuales': 450000, 'gastos_mensuales': 400000, 'deuda_total': 0, 'historial_crediticio': 'Excelente', 'dias_ultima_actualizacion': 5, 'deuda_document': 'Usuario ID: 793475\nDeuda Total: 0\nHistorial Crediticio: Excelente\nDescripción: El usuario con numero de I

In [24]:
query = "quiero 10 clientes con historial crediticio excelente pero con menores dias de inactividad "
result = qa({"query": query})
result

{'query': 'quiero 10 clientes con historial crediticio excelente pero con menores dias de inactividad ',
 'result': 'Lo siento, no tengo suficientes clientes con historial crediticio excelente y menos días de inactividad para recomendar. Los clientes que tengo disponibles ya han sido presentados y no cumplen con los requisitos solicitados.',
 'source_documents': [Document(page_content='Usuario_id: 423089\nSaldo_actual: 320000\nIngresos_mensuales: 300000\nGastos_mensuales: 300000\nDeuda_total: 0\nDias_ultima_actualizacion: 10\nCategories: Excelente\nDescription: El usuario con numero de ID 00423089 tiene un saldo actual de 320000$ e ingresos mesuales de 300000$. Contra eso, se registran gastos mensuales de 300000$ lo cual lleva a un neto de 0$, la deuda acumulada es 0$. El usuario tiene una clasificacion de historial crediticio Excelente. En terminos de actividad, se registra la ultima actualizacion el dia 2023-11-18, indica 10 dias de inactividad.', metadata={'@search.score': 10.830692

In [25]:
pip show langchain

Name: langchain
Version: 0.0.187
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/carolina/Descargas/nu0002_qa_apps_azure-main (copia)/venv/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: llama-index
Note: you may need to restart the kernel to use updated packages.
